In [1]:
import sqlite3
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# Connect to the SQLite database
conn = sqlite3.connect('../data_storage/spotify.db')

# SQL query to retrieve playlist tracks with track_uri, track_name, and pid
query = '''
    SELECT pt.pid, pt.track_uri, t.track_name
    FROM playlist_track pt
    JOIN track t ON pt.track_uri = t.track_uri
    WHERE pt.pid IN (SELECT pid FROM playlist LIMIT 2500)
'''

playlist_tracks = pd.read_sql_query(query, conn)
print(playlist_tracks)

conn.close()

co_occurrence_matrix = pd.crosstab(playlist_tracks['track_uri'], playlist_tracks['pid'])
print(co_occurrence_matrix)

n_components = min(co_occurrence_matrix.shape)  
print(n_components)
svd = TruncatedSVD(n_components=n_components, random_state=42)
song_embeddings = svd.fit_transform(co_occurrence_matrix)
print(song_embeddings)
# Display the shape of the resulting song embeddings
print("Shape of Song Embeddings:", song_embeddings.shape)



         pid                             track_uri  \
0       2093  spotify:track:000VZqvXwT0YNqKk7iG2GS   
1        371  spotify:track:000mA0etY38nKdvf1N04af   
2        182  spotify:track:000xQL6tZNLJzIrtIgxqSl   
3        813  spotify:track:000xQL6tZNLJzIrtIgxqSl   
4       1011  spotify:track:000xQL6tZNLJzIrtIgxqSl   
...      ...                                   ...   
165575  2355  spotify:track:7zxRMhXxJMQCeDDg0rKAVo   
165576  2444  spotify:track:7zxRMhXxJMQCeDDg0rKAVo   
165577   467  spotify:track:7zz1drChhd4hQBiGSnLRBZ   
165578  1006  spotify:track:7zzAWJ8aD50WH1EjGC2j45   
165579   454  spotify:track:7zzBEZBTJejWeL6EqWmCD9   

                              track_name  
0                                  Mercy  
1       If I Gave Myself To Someone Else  
2                         Still Got Time  
3                         Still Got Time  
4                         Still Got Time  
...                                  ...  
165575                          Some Way  
165576 

In [2]:
from sklearn.cluster import KMeans

num_clusters = 50  
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(song_embeddings)

valid_pids = co_occurrence_matrix.index.unique()
playlist_tracks_subset = playlist_tracks.drop_duplicates(subset='track_uri')

playlist_tracks_subset.reset_index(drop=True, inplace=True)
print("\nOriginal playlist_tracks DataFrame:")
print(playlist_tracks)

print("\nFiltered playlist_tracks_subset DataFrame:")
print(playlist_tracks_subset)

playlist_tracks_subset['cluster_label'] = cluster_labels

print(playlist_tracks_subset)

for cluster_id in range(num_clusters):
    print(f"\nCluster {cluster_id}:")
    tracks_in_cluster = playlist_tracks_subset[playlist_tracks_subset['cluster_label'] == cluster_id]['track_name']
    for i, track_name in enumerate(tracks_in_cluster):
        if i < 10:
            print(f"- {track_name}")
        else:
            break


Original playlist_tracks DataFrame:
         pid                             track_uri  \
0       2093  spotify:track:000VZqvXwT0YNqKk7iG2GS   
1        371  spotify:track:000mA0etY38nKdvf1N04af   
2        182  spotify:track:000xQL6tZNLJzIrtIgxqSl   
3        813  spotify:track:000xQL6tZNLJzIrtIgxqSl   
4       1011  spotify:track:000xQL6tZNLJzIrtIgxqSl   
...      ...                                   ...   
165575  2355  spotify:track:7zxRMhXxJMQCeDDg0rKAVo   
165576  2444  spotify:track:7zxRMhXxJMQCeDDg0rKAVo   
165577   467  spotify:track:7zz1drChhd4hQBiGSnLRBZ   
165578  1006  spotify:track:7zzAWJ8aD50WH1EjGC2j45   
165579   454  spotify:track:7zzBEZBTJejWeL6EqWmCD9   

                              track_name  
0                                  Mercy  
1       If I Gave Myself To Someone Else  
2                         Still Got Time  
3                         Still Got Time  
4                         Still Got Time  
...                                  ...  
165575       

C:\Users\joepa\AppData\Local\Temp\ipykernel_11148\681197809.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist_tracks_subset['cluster_label'] = cluster_labels


In [3]:
for cluster_id in range(num_clusters):
    print(f"\nCluster {cluster_id}:")
    tracks_in_cluster = playlist_tracks_subset[playlist_tracks_subset['cluster_label'] == cluster_id]['track_name']
    for i, track_name in enumerate(tracks_in_cluster):
        if i < 25:
            print(f"- {track_name}")
        else:
            break


Cluster 0:
- Satisfy You
- One More
- Don't Let Go (Love)
- Making Love (Into the Night)
- Unpretty
- Runnin' (Lose It All)
- Like I Do
- Again
- Stormy Monday
- A Change Is Gonna Come
- JAY Z Blue
- Love Is My Disease
- StreetLove
- Ta en bild / Lotus
- Fall For Your Type
- I Go Sailing
- Sky's The Limit (feat. 112) - 2014 Remastered Version
- Insight XX
- Bloodstream
- Babyfather
- Teach U a Lesson
- Addicted
- Strip For You
- Hey Lover
- I Love You

Cluster 1:
- U With Me?
- Bloodstream
- Interlude
- Behind the Light
- The Heathen
- How Great (feat. Jay Electronica & My cousin Nicole)
- Get Thy Bearings
- Crew Love
- Feel Good Inc
- Transits
- Amsterdam
- Everything
- Fantasy
- Shutdown
- Gang Related
- Take Me Home - Radio Edit
- Shades Of Blue
- Ten Crack Commandments - 2014 Remastered Version
- Favorite Color
- By Any Means
- Black Lip Bastard (Rmx) (feat. Black Hippy)
- Flex
- Frank's Track
- Hedron
- I Told You/ Another One

Cluster 2:
- Apologies Are For The Weak
- Blank Spac

In [4]:
import sqlite3
import pandas as pd
from gensim.models import Word2Vec

conn = sqlite3.connect('../data_storage/spotify.db')

query = '''
    SELECT pt.pid, pt.track_uri, t.track_name
    FROM playlist_track pt
    JOIN track t ON pt.track_uri = t.track_uri
    WHERE pt.pid IN (SELECT pid FROM playlist LIMIT 500000)
'''

playlist_tracks = pd.read_sql_query(query, conn)
conn.close()

playlists = playlist_tracks.groupby('pid')['track_uri'].apply(list).tolist()

model = Word2Vec(sentences=playlists, vector_size=100, window=5, min_count=1, sg=1)

track_embeddings = {track_uri: model.wv[track_uri] for track_uri in playlist_tracks['track_uri'].unique()}

example_track_uri = 'example_track_uri'
if example_track_uri in track_embeddings:
    print(f"Embedding for '{example_track_uri}':")
    print(track_embeddings[example_track_uri])

print("Shape of Song Embeddings:", len(track_embeddings), model.vector_size)


Shape of Song Embeddings: 1610661 100


In [5]:

import numpy as np
embeddings_array = np.array([track_embeddings[track_uri] for track_uri in playlist_tracks_subset['track_uri']])

num_clusters = 50

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings_array)

playlist_tracks_subset['cluster_label'] = cluster_labels

for cluster_id in range(num_clusters):
    print(f"\nCluster {cluster_id}:")
    tracks_in_cluster = playlist_tracks_subset[playlist_tracks_subset['cluster_label'] == cluster_id]['track_name']
    for i, track_name in enumerate(tracks_in_cluster):
        if i < 10:
            print(f"- {track_name}")
        else:
            break



Cluster 0:
- Nobody
- Dance With Me
- Spiritual
- I Don't Need A Man
- Bang Bang Bang
- Army
- Only Love
- True Love
- Catch Us If You Can
- Save Me

Cluster 1:
- Her
- Bodysnatchers
- Paper Trails
- I Like You
- Digital Kids
- Open
- Halfway Home
- Wildewoman
- The Worth Of The Wait
- & It Was U

Cluster 2:
- A Song For The Nameless
- Primitive (Richard Vission vs Luciana) - Original Mix
- If It Don't Take Two
- Lights Out
- Left Behind
- Mtc
- Calendar Pages
- Tactical Nuke
- Away
- Forgetting You

Cluster 3:
- Back Me Up
- Breaking News
- ??? (Where) [feat. Migos]
- Wasted (feat. Plies) - feat. Plies Album Version
- The Force
- Not My Bro
- The Bay to LA (feat. Murs)
- The Spiteful Chant
- I'm a Junkie
- Real

Cluster 4:
- Yams
- BEN CARSON
- Philosopher's Throne
- Let Me In
- 50 On My Wrist
- Raid (feat. 50 Cent & Pharrell)
- Love U Better (feat. Lil Wayne & The-Dream)
- Bodak Yellow
- Jumpman
- Know Better

Cluster 5:
- The One Who Really Loves You - Single Version
- One Summer N

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

song_names = [
    "Love Sosa",
    "Devil In A New Dress",
    "Rock The Boat",
    "I Say A Little Prayer",
    "Johnny B. Goode",
    "Roll Over Beethoven",
    "Can't Help Falling in Love",
    "Electric Feel",
    "Only Girl (In The World)",
    "Crew Love",
    "Everything In Its Right Place",
    "Karma Police"
]

selected_embeddings = []
selected_song_names = []

for name in song_names:
    if name in name_to_uri_map and name_to_uri_map[name] in track_embeddings:
        uri = name_to_uri_map[name]
        embedding = track_embeddings[uri]
        selected_embeddings.append(embedding)
        selected_song_names.append(name)
    else:
        print(f"Embedding not found for '{name}'")

selected_embeddings = np.array(selected_embeddings)

similarity_matrix = cosine_similarity(selected_embeddings)

print("Similarity Scores Matrix:")
for i, name1 in enumerate(selected_song_names):
    for j, name2 in enumerate(selected_song_names):
        similarity_score = similarity_matrix[i][j]
        print(f"{name1} vs {name2}: {similarity_score}")



Similarity Scores Matrix:
Love Sosa vs Love Sosa: 1.0
Love Sosa vs Devil In A New Dress: 0.3319993019104004
Love Sosa vs Rock The Boat: 0.8413376808166504
Love Sosa vs I Say A Little Prayer: 0.7963106632232666
Love Sosa vs Johnny B. Goode: 0.802086353302002
Love Sosa vs Roll Over Beethoven: 0.7482631206512451
Love Sosa vs Can't Help Falling in Love: 0.693295419216156
Love Sosa vs Electric Feel: 0.7223843932151794
Love Sosa vs Only Girl (In The World): 0.7744648456573486
Love Sosa vs Crew Love: 0.8062435388565063
Love Sosa vs Everything In Its Right Place: 0.7331420183181763
Love Sosa vs Karma Police: 0.7403231263160706
Devil In A New Dress vs Love Sosa: 0.3319993019104004
Devil In A New Dress vs Devil In A New Dress: 0.9999999403953552
Devil In A New Dress vs Rock The Boat: 0.27337324619293213
Devil In A New Dress vs I Say A Little Prayer: 0.4678381085395813
Devil In A New Dress vs Johnny B. Goode: 0.260274738073349
Devil In A New Dress vs Roll Over Beethoven: 0.38734573125839233
Devil